In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

def generate_dataset(num_examples):
    """Generate a dataset of Tic-Tac-Toe boards and labels"""
    X = np.zeros((num_examples, 9))
    y = np.zeros((num_examples, 9))
    z = np.zeros((num_examples))
    for i in range(num_examples):
        # Generate a random Tic-Tac-Toe board
        board = np.zeros((3, 3))
        player = 1
        move_count = 0
        while True:
            # Make a valid move
            empty_cells = np.argwhere(board == 0)
            if len(empty_cells) == 0:
                break
            idx = np.random.randint(len(empty_cells))
            row, col = empty_cells[idx]
            board[row][col] = player
            move_count += 1
            # Check for a win or a draw
            if move_count >= 5:
                if (np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or 
                    np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3):
                    # player 1 wins
                    break
                elif (np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or 
                      np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3):
                    # player 2 wins
                    break
                elif np.all(board != 0):
                    # game is a draw, but we don't want to include it in the dataset
                    break
            # Switch players after each move
            player = 1 if player == 2 else 2
        X[i] = board.flatten()
        # Calculate the label for the board
        if (np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or 
        np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3):
        # player 1 wins
          y[i] = np.array([1 if cell == 1 else 0 for cell in board.flatten()])
          z[i] = 1
        elif (np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or 
              np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3):
            # player 2 wins
            y[i] = np.array([1 if cell == 2 else 0 for cell in board.flatten()])
            z[i] = -1
        else:
            # game is a draw
            break
    return X, y, z


def print_board(board):
  """Print the Tic-Tac-Toe board"""
  print("  0 1 2")
  for i in range(3):
    print(i, end=" ")
    row = " ".join(["X" if cell == 1 else "O" if cell == 2 else " " for cell in board[i]])
    print(row)
   
def get_move(board, player, model, X):
    """Get the next move for the specified player using the model"""
    if player == 1:
        # Human player's turn
        while True:
            try:
                row = int(input("Enter row index: "))
                col = int(input("Enter column index: "))
                if row >= 0 and row < 3 and col >= 0 and col < 3 and board[row][col] == 0:
                    return row, col
                else:
                    print("Invalid move. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a valid row and column index.")
    else:
        # AI's turn
        # Generate predictions for all empty cells
        empty_cells = np.argwhere(board == 0)
        y_pred = model.predict(X[:len(empty_cells)])
        # Choose the cell with the highest predicted probability of winning for the AI
        idx = np.argmax(y_pred[:, player - 1])
        row, col = empty_cells[idx]
        return row, col

def play_game(model, X):
    """Play a game of Tic-Tac-Toe against the AI"""
    while True:
        board = np.zeros((3, 3))
        player = 1
        while True:
            print_board(board)
            row, col = get_move(board, player, model, X)
            board[row][col] = player
            if (np.all(board[0] == player) or np.all(board[1] == player) or np.all(board[2] == player) or 
                np.all(board[:,0] == player) or np.all(board[:,1] == player) or np.all(board[:,2] == player) or 
                np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player)):
                # player wins
                print(f"Player {player} wins!")
                break
            elif np.all(board != 0):
                # game is a draw
                print("The game is a draw.")
                break
            player = 1 if player == 2 else 2
        while True:
            play_again = input("Do you want to play again? (y/n) ")
            if play_again.lower() == "y":
                break
            elif play_again.lower() == "n":
                return
            else:
                print("Invalid input. Please enter 'y' or 'n'.")

# Create the model
model = Sequential()

# Add an LSTM layer with 32 units
model.add(LSTM(32, input_shape=(9, 1)))

# Add a fully-connected layer with 64 units and ReLU activation
model.add(Dense(64, activation='relu'))

# Add a final output layer with 9 units and sigmoid activation
model.add(Dense(9, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Generate the Tic-Tac-Toe dataset
X, y, z = generate_dataset(10000)

# Train the model
model.fit(X, y, epochs=10, batch_size=64)

# Play a game of Tic-Tac-Toe against the AI
play_game(model, X)

Epoch 1/10
157/157 [==============================] - 3s 8ms/step - loss: 0.6437 - accuracy: 0.1546
Epoch 2/10
157/157 [==============================] - 1s 8ms/step - loss: 0.6251 - accuracy: 0.0861
Epoch 3/10
157/157 [==============================] - 1s 8ms/step - loss: 0.5670 - accuracy: 0.0681
Epoch 4/10
157/157 [==============================] - 1s 7ms/step - loss: 0.4375 - accuracy: 0.1676
Epoch 5/10
157/157 [==============================] - 1s 7ms/step - loss: 0.3724 - accuracy: 0.2247
Epoch 6/10
157/157 [==============================] - 1s 7ms/step - loss: 0.3024 - accuracy: 0.3307
Epoch 7/10
157/157 [==============================] - 1s 8ms/step - loss: 0.2243 - accuracy: 0.4112
Epoch 8/10
157/157 [==============================] - 1s 7ms/step - loss: 0.1656 - accuracy: 0.4530
Epoch 9/10
157/157 [==============================] - 1s 7ms/step - loss: 0.1185 - accuracy: 0.4720
Epoch 10/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0857 - accuracy: 0.4759

KeyboardInterrupt: ignored